<a href="https://colab.research.google.com/github/amandatz/computational-linear-algebra/blob/main/Atividade2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 2

In [11]:
using LinearAlgebra

In [12]:
using Printf

function print_matrix(M)
    for i in 1:size(M, 1)
        for j in 1:size(M, 2)
            @printf("%10.4f ", M[i, j])
        end
        println()
    end
end


print_matrix (generic function with 1 method)

## Questão 1

In [13]:
function back_substitution(U, b)
    n = length(b)
    x = zeros(n)

    for i in n:-1:1
        sum_term = sum(U[i, j] * x[j] for j in i+1:n; init=0.0)
        x[i] = (b[i] - sum_term) / U[i, i]
    end

    return x
end

back_substitution (generic function with 1 method)

In [14]:
function forward_substitution(L, b)
    n = length(b)
    x = zeros(n)

    for i in 1:n
        sum_term = sum(L[i, j] * x[j] for j in 1:i-1; init=0.0)
        x[i] = (b[i] - sum_term) / L[i, i]
    end

    return x
end


forward_substitution (generic function with 1 method)

In [15]:
n = 4
U = triu(rand(n, n))
L = tril(rand(n, n))
b = rand(n)

x_back = back_substitution(U, b)
x_forward = forward_substitution(L, b)

println("U = ")
print_matrix(U)
println("Solução (back-substitution):")
print_matrix(x_back)

println("\nL = ")
print_matrix(L)
println("Solução (forward-substitution):")
print_matrix(x_forward)


U = 
    0.8477     0.0749     0.4431     0.5266 
    0.0000     0.3182     0.7747     0.7619 
    0.0000     0.0000     0.5532     0.2620 
    0.0000     0.0000     0.0000     0.9160 
Solução (back-substitution):
   -0.2915 
   -2.8283 
    1.3685 
    0.8845 

L = 
    0.1441     0.0000     0.0000     0.0000 
    0.1023     0.7242     0.0000     0.0000 
    0.6931     0.0534     0.0383     0.0000 
    0.2166     0.3333     0.4533     0.9798 
Solução (forward-substitution):
    4.2546 
    0.5506 
  -51.9738 
   23.7461 


## Questão 2

**Fatoração LU**

In [16]:
function flu(A)
    n = size(A, 1)
    L = Matrix{Float64}(I, n, n)
    U = copy(A)

    for k in 1:n-1
        for i in k+1:n
            L[i, k] = U[i, k] / U[k, k]
            U[i, k:n] -= L[i, k] * U[k, k:n]
        end
    end

    return L, U
end


flu (generic function with 1 method)

**Fatoração de Cholesky**

In [17]:
function fchol(A)
    n = size(A, 1)
    L = zeros(n, n)

    for i in 1:n
        for j in 1:i
            sum_k = dot(L[i, 1:j-1], L[j, 1:j-1])
            if i == j
                L[i, j] = sqrt(A[i, i] - sum_k)
            else
                L[i, j] = (A[i, j] - sum_k) / L[j, j]
            end
        end
    end

    return L
end

fchol (generic function with 1 method)

In [38]:
function solve_system(n)
    A = diagm(0 => fill(4.0, n),
              1 => fill(-1.0, n-1), -1 => fill(-1.0, n-1),
              2 => fill(-1.0, n-2), -2 => fill(-1.0, n-2))
    b = A[:, n]

    println("\n============================")
    println("n = $n")

    t_lu = @elapsed begin
        L, U = flu(A)
        y = forward_substitution(L,b)
        x_lu = back_substitution(U,y)
    end

    t_chol = @elapsed begin
        L = fchol(A)
        y = forward_substitution(L,b)
        x_chol = back_substitution(L',y)
    end

    x_true = zeros(n)
    x_true[end] = 1 # por que??? explicar
    err_lu = norm(x_lu - x_true) / norm(x_true)
    err_chol = norm(x_chol - x_true) / norm(x_true)

    println("== Fatoração LU ==")
    println("Tempo: ", t_lu, " segundos")
    println("Erro relativo: ", err_lu)

    println("\n== Fatoração de Cholesky ==")
    println("Tempo: ", t_chol, " segundos")
    println("Erro relativo: ", err_chol)
end

solve_system (generic function with 1 method)

In [39]:
for n in [10, 100, 200, 400]
    solve_system(n)
end


n = 10
== Fatoração LU ==
Tempo: 1.9234e-5 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 1.1387e-5 segundos
Erro relativo: 0.0

n = 100
== Fatoração LU ==
Tempo: 0.00315154 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 0.001265525 segundos
Erro relativo: 0.0

n = 200
== Fatoração LU ==
Tempo: 0.028548265 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 0.008029958 segundos
Erro relativo: 9.720503087223581e-16

n = 400
== Fatoração LU ==
Tempo: 0.34887259 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 0.079356894 segundos
Erro relativo: 1.6024819772089442e-15


## Questão 3

## Questão 4

In [21]:
n= 6

A = 0.1 * ones(n, n)
for i in 1:n
    A[i, i] = 1
end

L = fchol(A)
println("Matriz L (Fator de Cholesky):")
print_matrix(L)

Matriz L (Fator de Cholesky):
    1.0000     0.0000     0.0000     0.0000     0.0000     0.0000 
    0.1000     0.9950     0.0000     0.0000     0.0000     0.0000 
    0.1000     0.0905     0.9909     0.0000     0.0000     0.0000 
    0.1000     0.0905     0.0826     0.9874     0.0000     0.0000 
    0.1000     0.0905     0.0826     0.0760     0.9845     0.0000 
    0.1000     0.0905     0.0826     0.0760     0.0703     0.9820 
